In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains
import os.path
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys
from vogueSnack import Sourcing, Uploading, Tool
import tkinter as tk
from selenium.webdriver.edge.options import Options as EdgeOptions
root = tk.Tk()
options = EdgeOptions()
# Creating and placing the result Text widget
result_text = tk.Text(root, height=10, width=50)
result_text.pack()
# Configure the tag for red text
result_text.tag_configure("red", foreground="red")

driver = webdriver.Edge(options=options)
EdgeTool = Tool(driver, result_text)
EdgeSourcing = Sourcing(driver, tool=EdgeTool)
EdgeUploading = Uploading(driver, tool=EdgeTool)


In [37]:
prd_list_coupang_df = EdgeSourcing.downloadChecker('/Users/papag/Downloads', "Coupang_detailinfo", first_phase=False)
prd_code_coupang_series = prd_list_coupang_df.iloc[:,1]
prd_code_coupang_df = prd_code_coupang_series.to_frame(name='prd_code')
prd_code_coupang_df.insert(prd_code_coupang_df.columns.get_loc('prd_code') + 1, 'isExist', False)
# prd_code_coupang_df.to_csv('prd_code_coupang_df.csv')
prd_code_coupang_df.dropna(inplace=True)
prd_code_coupang_df.reset_index(drop=True, inplace=True)

[+] Coupang_detailinfo downloaded successfully. Start reading.


c:\Users\papag\OneDrive\src\Projects\vogueSnack\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


[+] Coupang_detailinfo read with excel successfully.


c:\Users\papag\OneDrive\src\Projects\vogueSnack\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\papag\OneDrive\src\Projects\vogueSnack\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [38]:
prd_code_coupang_df.drop(prd_code_coupang_df.index[0:14], inplace=True)
prd_code_coupang_df.reset_index(drop=True, inplace=True)


,prd_code,isExist
0,CH2286906 심플 노트북 2way백,False
1,CH2286955 여성 경량 백팩,False
2,CH2287015 남성 옥스포드 백팩,False
3,CH2287015 남성 옥스포드 백팩,False
4,CH2291450 패션 생활 방수 깔끔한 디자인 스러운 여성 고급 백팩,False
...,...,...
2942,CH2258349 스카로 야구공 청소기 야구공 자우개 로봇 기계 로봇청소기,False
2943,CH1881688 신일8000 LED조명 듀얼 회전 무선 물걸레 청소기 무선청소기,False
2944,CH1768717 분더바 샴푸바,False
2945,CH2271415 거품 바디 워시 플로럴 향 좋은 등드름 약산성 클렌져,False


In [40]:
prd_code_coupang_df['prd_code'] = prd_code_coupang_df['prd_code'].str.split().str[0]
prd_code_coupang_df

,prd_code,isExist
0,CH2286906,False
1,CH2286955,False
2,CH2287015,False
3,CH2287015,False
4,CH2291450,False
...,...,...
2942,CH2258349,False
2943,CH1881688,False
2944,CH1768717,False
2945,CH2271415,False


In [43]:
prd_name_list = pd.read_csv('coupang_suspended_prd.csv')
prd_name_list

,prd_code
